In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np 
from vqls import *
from utils import *


In [27]:
func_out = {'sigmoid': sigmoid_t,'tanh': tanh_t,'elu': elu_t, 'relu': relu_t, 'sin':sin_m}
func_list = list(func_out.keys())
func_dict = {'sigmoid': .0,'tanh': 1.0,'elu':.12, 'relu':.0, 'sin':.0}

pnq = {'3':(3,8),
        '4':(4,16)}
columns=['function', 'nq', 'knot', 'training cost', 'time(s)', 'weights', 'RSS_h', 'RSS_q' ]
tot = {}
for flab, fun in func_out.items():
    for nq, knot in  pnq.values():
        file_path = f"results_{flab}_{nq}.json"
        df = pd.read_json(file_path).drop(['Condition number', 'norm(yk)', 'training_cost', 'exe_time', 'in_train_weights', 'RSS_h'], axis =1 )
        tot[f"{flab}{nq}"] = df
tot

{'sigmoid3':        RSS_q  seed
 0   3.963544     2
 1   9.433116    27
 2   9.426826    70
 3   0.276599    52
 4   4.145746    22
 5   9.409706    72
 6   3.943581    60
 7   4.357991     7
 8   3.975778    67
 9   4.098747     1
 10  3.987636    54
 11  0.275849    89
 12  0.268566    28
 13  3.949084    20
 14  4.165525     4
 15  4.205331     0
 16  9.446836    51
 17  3.945904    81
 18  3.961218    78
 19  4.137898    42
 20  4.145746    22
 21  9.421849     5
 22  0.275371    44
 23  3.898131     9
 24  9.446836    51,
 'sigmoid4':        RSS_q  seed
 0   1.518124     7
 1   7.224090    26
 2   6.982597    50
 3  14.029737    48,
 'tanh3':        RSS_q  seed
 0   3.731439    74
 1   8.437812    97
 2   3.606715    61
 3   3.305360    83
 4   8.432703    79
 5   3.115177     1
 6   3.768617    99
 7   8.491775    32
 8   8.437812    97
 9   8.446650    49
 10  8.439313    72
 11  8.434177    43
 12  3.381518    22
 13  0.127738    56
 14  8.444673    48
 15  0.108705    12
 16  

,function,nq,knot,training cost,time(s),weights,RSS_h,RSS_q
0,sigmoid,3,8,"[0.43695497240000003, 0.30950764070000003, 0.2...",18635.241470,"[2.7451074065, 2.6239581654, 0.002763550600000...",0.466814,0.268566
1,sigmoid,4,16,"[0.436504891, 0.3373845206, 0.2353528699, 0.21...",139828.309502,"[1.5756719111000002, 3.1864332824, 3.007082071...",2.059185,1.518124
2,tanh,3,8,"[0.4814711189, 0.3728673488, 0.2810626613, 0.3...",15437.758282,"[0.1933855528, 3.1459687196, 1.2322714464, 1.6...",0.254278,0.108705
3,tanh,4,16,"[0.4577258524, 0.4295029442, 0.4788074825, 0.3...",170833.033432,"[3.0146464492, 2.4547842698, 2.597904879999999...",1.430799,0.977391
4,elu,3,8,"[0.47746529130000004, 0.3872581175, 0.28119417...",6669.839297,"[-0.0113545294, 2.6037647359, 0.68391100550000...",0.015660,0.114745
5,elu,4,16,"[0.4901639498, 0.4618650857, 0.448412788300000...",90587.401667,"[3.1883845824, 1.9524916928, 0.467479085200000...",0.143556,0.068217
6,relu,3,8,"[0.498158352, 0.41149152790000004, 0.369324353...",6817.104706,"[4.5202400282, 3.1356767902, 3.3606822234, -0....",0.179513,0.376055
7,relu,4,16,"[0.4771129396, 0.4542505499, 0.4495125291, 0.4...",120942.523607,"[0.5433257203, 3.4601980874000002, 2.413156998...",4.669781,5.329755
8,sin,3,8,"[0.42982478430000004, 0.427129291, 0.460793889...",6868.388830,"[3.9879893098, 0.26118481320000003, 0.58592689...",0.650911,0.401280
9,sin,4,16,"[0.39028068400000004, 0.3567349823, 0.49173851...",132477.144560,"[2.0227996589, 1.1522648395, 2.3841853205, 0.7...",11.478868,13.059758


In [12]:
dk= df.drop(['weights','time(s)','training cost','RSS_h'], axis=1)
#dk['RSS_q'] = dk['RSS_q']/dk['knot']
dk["mse"] = rss
print(dk.pivot(index='function',columns='nq', values=['mse','RSS_q']).to_markdown())

| function   |   ('mse', 3) |   ('mse', 4) |   ('RSS_q', 3) |   ('RSS_q', 4) |
|:-----------|-------------:|-------------:|---------------:|---------------:|
| elu        |     0.119763 |    0.0652962 |       0.114745 |      0.0682175 |
| relu       |     0.216811 |    0.069185  |       0.376055 |      5.32975   |
| sigmoid    |     0.182906 |    0.30803   |       0.268566 |      1.51812   |
| sin        |     0.223964 |    0.312243  |       0.40128  |     13.0598    |
| tanh       |     0.116568 |    0.247158  |       0.108705 |      0.977391  |


In [8]:
flab = 'relu'
fun = relu_t
lower = 0. 
upper = 1. 
f_i = func_dict[flab]
scaled=False    
label = flab
n_step = el.knot
x = np.arange(lower, upper + .03, (upper-lower)/n_step).tolist() 
xx = np.linspace(lower, upper, n_step) ##inputs sampling in the interval 0,1
y = [fun(value,f_i) for value in x]

tck=splrep(x,y,k=1) #coeffs
norm = np.linalg.norm(y)
y = y / norm
ax[i].plot(np.linspace(0,1,len(y)),y, label=f"Original function scaled {n_step}", color = '#5c4d7d',linewidth=2.5)

ax[i].axhline(0, color="#222222")
matrix,vector,v_norm = GeneralizedVQS_System(n_step,label,x,xx,scaled=scaled)

vqls_circuit = VQLS(matrix,v_norm,el['nq'],opt='COBYLA') 

c = np.linalg.solve(matrix,vector)

y_c=np.dot(matrix,c) #classic
ax[i].plot(np.linspace(0,1,len(y_c)),y_c, label="classical spline", color = '#1780a1',linewidth=2.5)

y_fq=[]
q = vqls_circuit.solution(el['weights']).real
y_q=np.dot(matrix,q) #hybrid
                #Quantum

for a in matrix:
    y_fq.append(vqls_circuit.direct_prod2(el['weights'],a))
rss.append(math.sqrt(np.square(np.subtract(y_c,y_fq)).mean()))


In [9]:
y_fq

[tensor(-0.24159205, requires_grad=True),
 tensor(-0.27983433, requires_grad=True),
 tensor(-0.4385123, requires_grad=True),
 tensor(-0.47483765, requires_grad=True),
 tensor(-0.51361569, requires_grad=True),
 tensor(-0.49873931, requires_grad=True),
 tensor(-0.40872349, requires_grad=True),
 tensor(-0.30831969, requires_grad=True),
 tensor(-0.15331156, requires_grad=True),
 tensor(-0.07351244, requires_grad=True),
 tensor(-0.01786041, requires_grad=True),
 tensor(-0.0303805, requires_grad=True),
 tensor(-0.03717123, requires_grad=True),
 tensor(-0.15253645, requires_grad=True),
 tensor(-0.16465614, requires_grad=True),
 tensor(-0.15439434, requires_grad=True)]